![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

### Итоговое задание Стрибук Иван (DSPR-37)
### по Проекту 3. О вкусной и здоровой пище 
####  Юнит 3. Введение в машинное обучение (отредактирован 26.07.2021)
---

# import

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import os
from sklearn.model_selection import train_test_split
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [13]:
from datetime import datetime, timedelta
import re

In [14]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [15]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [16]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

pd.set_option('display.max_columns', 200)  # смотреть удобно таблицы

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/sf-dst-restaurant-rating//main_task.csv'

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1  # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
# в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями
df_test['Rating'] = 0

data = df_test.append(df_train, sort=False).reset_index(
    drop=True)  # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

In [ ]:
# Restaurant_id
data['code_Restaurant_id'] = data['Restaurant_id'].apply(
    lambda x: float(x[3:]))

Визуальный осмотр показал, что код Restaurant_id очень сильно похож на Ranking в data_train.
Надо проверить корреляцию и при необходимости удалить 'code_Restaurant_id'

In [ ]:
rest_id_corr = data[['code_Restaurant_id', 'Ranking']]

In [ ]:
sns.pairplot(rest_id_corr, kind='reg')

In [ ]:
# Используем для наглядности матрицу корреляций:
rest_id_corr.corr()

In [ ]:
# вспоминаем Резюме по критерию code_Restaurant_id. Удаляем так как была гипотеза о корреляции с Ranking
data.drop(['code_Restaurant_id'], axis=1, inplace=True, errors='ignore')

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# вынесем пропуски в данных отдельным параметром
data['NoR NAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
# глянем в статистику
data['Number of Reviews'].describe()

Медиана и средняя сильно разняться, это значит, что в параметре много выбросов. для более детальной картины можно глянуть статистику по городам.

In [ ]:
data['City'].value_counts().sort_index(ascending=True)

In [ ]:
data.groupby(['City'])['Number of Reviews'].agg(
    ['mean', 'median']).plot(kind='bar', figsize=[20, 3], grid=True)

In [ ]:
# будем заполнять пропуски медианой по городам, она в данном случае более объективно представляет информацию

gb_city = round(data.groupby(['City'])[
                'Number of Reviews'].median()).to_dict()  # создаем словарь
# заменяем пропуски, применяя словарь
data['Number of Reviews'].fillna(data['City'].map(gb_city), inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
# перенесено после формирования признаков с использованием колонки City
#data = pd.get_dummies(data, columns=['City'], dummy_na=True)

In [ ]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [ ]:
data['Price Range'].value_counts()

In [ ]:
# и кол-во пропусков
display(data['Price Range'].isna().sum())

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [ ]:
# Ваша обработка 'Price Range'
# 23041 ресторанов это более 70% из заполненной информации имеют средний параметр цены
# поэтому заполняем пропуски двойкой (2)
data['Price Range'].fillna('$$ - $$$', inplace=True)
# заполним значения в переменной.
data['Price Range'].replace({'$': 1, '$$ - $$$': 2, '$$$$': 3}, inplace=True)

In [ ]:
# посмотрим статисику
display(data['Price Range'].describe())

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [ ]:
data['Cuisine Style'].value_counts(dropna=False).head(20)

In [ ]:
data[['City', 'Cuisine Style']].groupby('City').describe()

In [ ]:
data[['City', 'Cuisine Style']].groupby(
    'City')['Cuisine Style'].value_counts()['London'].head(10)

In [ ]:
# тут ваш код на обработку других признаков

# Заполняем пропуски для обзоров пустой строкой
data['Reviews'].fillna('', inplace=True)

# заполняем пропуски для видов кухни
data['Cuisine Style'].fillna('Unknown', inplace=True)

# Создаем признак для видов кухни типа list, предварительно убрав лишние символы
data['Cuisine_Style_List'] = data['Cuisine Style'].str.replace(
    "[", '').str.replace("]", '').str.replace("'", "").str.replace(", ", ",")
data['Cuisine_Style_List'] = data['Cuisine_Style_List'].str.split(',')

# Определяем кол-во видов кухни для ресторана
data['Cuisine_Count'] = data['Cuisine_Style_List'].apply(lambda x: len(x))

In [ ]:
# Заметно, что чем более разнообразна кухня, тем менше рейтинг ресторана.
sns.boxplot(x='Cuisine_Count', y='Ranking', data=data)

In [ ]:
data.head(5)

In [ ]:
# функция для извлечения дат отзывов
def get_dates(value):
    pat = re.compile("\d\d/\d\d/\d\d\d\d")
    review_list = pat.findall(value)
    return review_list

# функция для определения минимальной даты


def get_min_date(value):
    # Заполняем 0 для пустых значений по умолчанию, позднее его заменим в признаке
    res = 0
    if len(value) > 1:
        res = min(datetime.strptime(
            value[0], "%m/%d/%Y"), datetime.strptime(value[1], "%m/%d/%Y"))
    elif len(value) == 1:
        res = datetime.strptime(value[0], "%m/%d/%Y")
    return res

# функция для определения максимальной даты


def get_max_date(value):
    # Заполняем 0 для пустых значений по умолчанию, позднее его заменим в признаке
    res = 0
    if len(value) > 1:
        res = max(datetime.strptime(
            value[0], "%m/%d/%Y"), datetime.strptime(value[1], "%m/%d/%Y"))
    elif len(value) == 1:
        res = datetime.strptime(value[0], "%m/%d/%Y")
    return res

# функция для извлечения слов из отзывов


def get_words(value):
    # слова с not
    pat = re.compile('[Nn]ot [A-Za-z]+')
    review_list_n = pat.findall(value)
    for word in review_list_n:
        value = value.replace(word, '')

    pat = re.compile('[A-Za-z]+')
    review_list = pat.findall(value)
    review_list += review_list_n
    review_list_cor = []
    # переводим все слова в строчные буквы
    for word in review_list:
        review_list_cor.append(word.lower())
    return review_list_cor

In [ ]:
data.info()

In [ ]:
# Собираем слова из отзывов в отдельный признак
data['Reviews_Words'] = data['Reviews'].apply(get_words)

# определяем даты для отзывов
data['Reviews_Dates'] = data['Reviews'].apply(get_dates)
# минимальная дата отзыва
data['Reviews_Min_Date'] = data['Reviews_Dates'].apply(get_min_date)
# максимальная дата отзыва
data['Reviews_Max_Date'] = data['Reviews_Dates'].apply(get_max_date)

# Определяем минимальную дату в данных и заполняем ей 0-е (пустые) значения
tot_min_date = min(data[data['Reviews_Min_Date'] != 0]['Reviews_Min_Date'])
data['Reviews_Min_Date'] = data['Reviews_Min_Date'].replace(0, tot_min_date)
data['Reviews_Max_Date'] = data['Reviews_Max_Date'].replace(0, tot_min_date)


# разница между максимальной и минимальной датой отзыва
data['Reviews_Dates_diff'] = data['Reviews_Max_Date'] - data['Reviews_Min_Date']
data['Reviews_Dates_diff'] = data['Reviews_Dates_diff'].dt.days
# разница между максимальной датой отзыва и последней датой отзыва в данных
data['Reviews_Max_Date_diff'] = data['Reviews_Max_Date'].max() - \
    data['Reviews_Max_Date']
data['Reviews_Max_Date_diff'] = data['Reviews_Max_Date_diff'].dt.days

In [ ]:
# Определяем относительный ранг для ресторана внутри города
data1 = data.groupby(['City'])['Ranking'].max().to_frame().reset_index()
data1.columns = ['City', 'City_N_Rank']
data = data.merge(data1, on='City', how='left')
data['Ranking_rel'] = data['Ranking']/data['City_N_Rank']

In [ ]:
data.head(5)

In [ ]:
data.Reviews[:30]

In [ ]:
# смотрим наиболее часто встречающиеся в отзывах слова
data['Reviews_Words'].explode().value_counts()[0:35]

In [ ]:
# Экспертным методом определяем список слов из часто встречаюшихся, котрые могут влиять на оценку
# Как минимум проблема - отсутствие связки not со словами - теряется возомжный смысл
key_words = ['good', 'great', 'nice', 'best', 'excellent', 'but',
             'delicious', 'friendly', 'lovely', 'amazing', 'tasty', 'little']

In [ ]:
# функция для определения набора ключевых слов в отзывах с учетом кол-ва повторов
def num_key_words(value):
    # слова с not
    new_list = []
    for word in value:
        if word in key_words:
            new_list.append(word)
    return new_list

In [ ]:
data['key_words_n'] = data['Reviews_Words'].apply(num_key_words)

In [ ]:
data.head(5)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
# было выше в исходном варианте, перенесено для формирования данных с помощью признака City
data = pd.get_dummies(data, columns=['City'], dummy_na=True)

In [ ]:
# Добавляем также dummies признак для типов кухни
df1 = data[['Cuisine_Style_List']].copy()
df1 = pd.get_dummies(df1['Cuisine_Style_List'].explode(),
                     prefix='cuis').sum(level=0)
data = pd.concat([data, df1], axis=1)

In [ ]:
# Добавляем также dummies признак для ключевых слов в отзыве
df1 = data[['key_words_n']].copy()
df1 = pd.get_dummies(df1['key_words_n'].explode(), prefix='w').sum(level=0)
data = pd.concat([data, df1], axis=1)

In [ ]:
data.sample(5)

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [ ]:
# распределение в ранкинге
plt.rcParams['figure.figsize'] = (12, 6)
data['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
# Построим боксплоты для рейтинга
fig, ax = plt.subplots(figsize=(15, 10))
sns.boxplot(x='City', y='Rating', data=df_train)
plt.xticks(rotation=45)
ax.set_title('Boxplot for City')
plt.show()

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
# Распределение в Лондоне
df_train['Ranking'][df_train['City'] == 'London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

# Получается, что Ranking имеет нормальное распределение,
# просто в больших городах больше ресторанов, из-за мы этого имеем смещение

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

In [ ]:
plt.rcParams['figure.figsize'] = (10, 60)
sns.heatmap(data.query('sample == 1').corr()[
            'Rating'].dropna().to_frame(), cmap="YlGnBu", annot=True)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# #Этот фрагмент не используется - все данные из исходной части обработки

# # на всякий случай, заново подгружаем данные
# df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
# df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
# df_train['sample'] = 1 # помечаем где у нас трейн
# df_test['sample'] = 0 # помечаем где у нас тест
# df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

# data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
# data.info()

In [ ]:
# def preproc_data(df_input):
#     '''includes several functions to pre-process the predictor data.'''

#     df_output = df_input.copy()

#     # ################### 1. Предобработка ##############################################################
#     # убираем не нужные для модели признаки
#     df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)


#     # ################### 2. NAN ##############################################################
#     # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
#     df_output['Number of Reviews'].fillna(0, inplace=True)
#     # тут ваш код по обработке NAN
#     # ....


#     # ################### 3. Encoding ##############################################################
#     # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
#     df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
#     # тут ваш код не Encoding фитчей
#     # ....


#     # ################### 4. Feature Engineering ####################################################
#     # тут ваш код не генерацию новых фитчей
#     # ....


#     # ################### 5. Clean ####################################################
#     # убираем признаки которые еще не успели обработать,
#     # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
#     object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
#     df_output.drop(object_columns, axis = 1, inplace=True)

#     return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
#df_preproc = preproc_data(data)
# df_preproc.sample(10)

In [ ]:
# df_preproc.info()

In [ ]:
# data['sample']

In [ ]:
data.info()

In [ ]:
data.columns[:30]

In [ ]:
# Берем данные из предварительной обработки
df_preproc = data.drop(['Restaurant_id', 'Cuisine Style', 'Reviews', 'URL_TA', 'ID_TA', 'Cuisine_Style_List', 'Reviews_Words',
                        'Reviews_Dates', 'Reviews_Min_Date', 'Reviews_Max_Date', 'key_words_n'], axis=1)

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics  # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(
    n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

заметим, что рейтинг имеет шаг 0.5. Модель предсказывает рейтинг без учёта этой детали.
Если вручную округлить результаты предсказания, то их качество улучшится.

In [ ]:
y_pred_round = np.round(y_pred * 2)/2
display(y_pred_round)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MAE considering round:', metrics.mean_absolute_error(y_test, y_pred_round))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10, 10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
# Проверяем корреляцию важных переменных
df_temp = data.loc[data['sample'] == 1, list(
    feat_importances.nlargest(15).index[0:15])]
plt.rcParams['figure.figsize'] = (12, 6)
ax = sns.heatmap(df_temp.corr(), annot=True, fmt='.2g')
i, k = ax.get_ylim()
ax.set_ylim(i+0.5, k-0.5)

In [ ]:
list_temp = list(feat_importances.nlargest(15).index[[9, 10]])
display(df_temp[list_temp].corr())

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.shape

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
# округляем до 0,5 как в фактических оценках
# (predict_submission * 2).round(0) / 2
predict_submission = np.round(predict_submission * 2)/2
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
